<a href="https://colab.research.google.com/github/lucascodinglab/Smieja_Neuronal_Network/blob/main/Smieja_Neuronal_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)

Mounted at /content/drive/


In [2]:
import statsmodels.api as sm
import numpy as np
import pandas as pd
import math

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
import statsmodels.api as sm
import tqdm
import sklearn
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.mixture import GaussianMixture
from scipy.stats import multivariate_normal
from tensorflow.python.ops.math_ops import sqrt_grad
import tensorflow as tf
import warnings
import keras
import torch
from tensorflow.keras.utils import to_categorical

In [3]:
class Model():
    
    def __init__(self, weights, biases, alpha=0.00):
        self.weights = weights
        self.biases = biases
        self.num_layers = len(weights)
        self.alpha = alpha 
    
    #propagates a list of points through the model; each hidden layer is followed by an 
    #activation layer by a leaky ReLu with self.alpha as parameter 
    #need to write a function for PropagationBase.py which uses activation function already implemented 
    #points must be numpy arrays of size >=2
    def propagate(self, point, start_layer):
        for i in range(start_layer, self.num_layers):
            point = np.matmul(self.weights[i], point) + self.biases[i]
           
            #if i in range(0, self.num_layers-1):
            if i < self.num_layers - 1:
                for j in range(0, self.weights[i].shape[0]):
                    if point[j]<0:
                        point[j] = self.alpha * point[j]
        
            
        
        return point
        
def loadModel(file, alpha=0.00):
    
    param_dict = torch.load(file) #"NN1.pt"
    w = []
    b= []
    num_layers = len({int(item.split("_")[1]) for item in param_dict})
    #print(num_layers)
    for i in range(num_layers):
            w.append(param_dict["weights_" + str(i)].numpy())
            b.append(param_dict["bias_" + str(i)].numpy())

    model = Model(w, b, alpha=alpha)
    return model 

def generate_missing_values(complete_data, p):
    shape = complete_data.shape
    y = complete_data.copy()
    missing = np.random.binomial(1, p, shape)
    y[missing.astype('bool')] = np.nan
    #print(y, complete_data)
    return y

def fit_distribution(x, components_for_GMM, type_of_distribution="normal"): # x sind alle daten und vollständig 
    if type_of_distribution == "normal":
        distr_on_x = GaussianMixture(n_components = 1, covariance_type= "diag").fit(x) 
    if type_of_distribution == "GMM":
        distr_on_x = GaussianMixture(n_components = components_for_GMM, covariance_type= "diag").fit(x) 
    return distr_on_x  

def nr_new(x):
    one_diff_sqrt = np.divide(1,np.sqrt(2*math.pi))
    exp = np.exp(-np.divide(x**2, 2))
    erf_input = np.divide(x, np.sqrt(2))
    erf = math.erf(erf_input)
    erf_part = np.divide(x,2) * (1+erf)
    return one_diff_sqrt * exp + erf_part   

def relu_gmm(mu, sigma, w, p, b): # calculates relu for every neuron of the first hidden layer()
    wT = np.transpose(w)  
    sqrt_ = np.sqrt(np.dot(np.multiply(wT, sigma), w))
    part_1 = p * sqrt_
    x = np.divide(wT @ mu + b, sqrt_)
    part_2 = nr_new(x)

    relu_f = part_1 * part_2

    return relu_f

def regularization(gamma, n_components, cov, mu, x_certain):
    reg_weights = []
    num_dim_certain = len(x_certain)
    for component in range(0,n_components):  
        cov_component = cov[0][component]
        mu_component = mu[component]
        sub_part_factors = 1
        sub_part_sum = 0
        for dim in range(0,num_dim_certain):
            sub_part_factors *= (gamma + cov_component[dim])**0.5  
            sub_part_sum += ( (1 / gamma + cov_component[dim]) * (mu_component[0][dim] - x_certain[dim])**2)
            # print("mu_component: ",mu_component[0][dim])
            # print("x_certain: ",x_certain[dim])
            
            
        part_1 = 1 / (2 * np.pi**(num_dim_certain / 2) * sub_part_factors ) 
        part_2 = math.exp(-0.5 * sub_part_sum)
        C = part_1 * part_2
        # print("sub_part_sum: ",sub_part_sum)
        # print("part_2: ", part_2)
        
        reg_weights.append(C)
    
    return reg_weights

def GMM_missing_generation(y, distr,n_components, gamma, type_of_distribution="GMM"):
    if type_of_distribution == "normal":
        pass
    if type_of_distribution == "GMM":
        missing_values = []
        certain_mean = []
        certain_cov = []
        for i, yi in enumerate(y):
            if math.isnan(yi):
                missing_values.append(True)
            else:
                missing_values.append(False)
                certain_mean.append(yi)
                certain_cov.append(0)
        if True in missing_values:
            if False in missing_values:

                GMM_missing = GaussianMixture()
                GMM_missing.means_ = np.array([[j[missing_values]] for j in distr.means_]) 
                GMM_missing.covariances_ = np.array([[j[missing_values] for j in distr.covariances_]])  
                
                certain_values_true = [not elem for elem in missing_values]  
               
                GMM_certain = GaussianMixture()
                GMM_certain.means_ = np.array([[j[certain_values_true]] for j in distr.means_])
                GMM_certain.covariances_ = np.array([[j[certain_values_true] for j in distr.covariances_]])
                # print(GMM_certain.covariances_)
                # print("Missing Means:", GMM_missing.means_, "\n Missing Covs:", GMM_missing.covariances_)
                nv_certain = []
                C = regularization(gamma = gamma, n_components = n_components, cov = GMM_certain.covariances_, mu = GMM_certain.means_,x_certain = certain_mean)
                for i in range(distr.n_components):
                    nv_certain.append(multivariate_normal.pdf(y[certain_values_true], mean = GMM_certain.means_[i][0], cov = np.diagflat(GMM_certain.covariances_[0][i])))
                nv_certain = np.array(nv_certain)
                # print("NV_certain ", nv_certain)
                # print("Y Values: ", GMM_certain.covariances_[0][4])
                # print("vor Division: ", distr.weights_, C)
                GMM_missing.weights_ = distr.weights_ * C
                GMM_missing.weights_ = GMM_missing.weights_/np.sum(GMM_missing.weights_)
                # print("nach Division: ", GMM_missing.weights_)
                GMM_total = GaussianMixture()
                dimensions = n_components
                GMM_total_mean = []
                GMM_total_cov = []
                for d in range(0,dimensions):
                    GMM_mean_arr = []
                    GMM_covariance_arr = []
                    counter_gmm = 0  
                    counter_arr = 0 
                    for i, yi in enumerate(y):
                          if np.isnan(yi):
                              GMM_mean_arr.append(GMM_missing.means_[d][0][counter_gmm])
                              GMM_covariance_arr.append(GMM_missing.covariances_[0][d][counter_gmm])
                              counter_gmm+=1
                          else:
                              GMM_mean_arr.append(certain_mean[counter_arr])
                              GMM_covariance_arr.append(certain_cov[counter_arr])
                              counter_arr+=1
                    #append after every dimension
                    GMM_total_mean.append(GMM_mean_arr)
                    GMM_total_cov.append(GMM_covariance_arr)
                  
                matrix_mean = np.array(GMM_total_mean)
                matrix_cov = np.array(GMM_total_cov)
                GMM_total.means_ = matrix_mean  #np.reshape(matrix_mean, (178, 4))
                GMM_total.covariances_ = matrix_cov
                GMM_total.weights_ = GMM_missing.weights_
                # print("Ergebnis von GMM_missing_generation:", GMM_missing.weights_)

                return GMM_total
            else:
                return distr
        else:
            return y

def fill_gmm(X, df, gamma):
    df_distr = df.drop(columns=["y"],axis=1)
    distr = fit_distribution(df_distr, components_for_GMM = 5, type_of_distribution="GMM")
    GMM_missing= GMM_missing_generation(X, distr, 5, gamma, type_of_distribution="GMM")

    return GMM_missing

def iterate_gmm_neuron(x, df, n_components, w, b, gamma):
    relu_sum = 0
    #get distribution
    df_distr = df.drop(columns=["y"],axis=1)
    distr = fit_distribution(df_distr,n_components,"GMM")
    #get GMM for missing values
    GMM_total = GMM_missing_generation(x,distr,n_components,gamma, type_of_distribution="GMM")
    #get Parameters from GMM
    mu_arr = GMM_total.means_
    #print(mu_arr)
    sigma_arr = GMM_total.covariances_
    #print(sigma_arr)
    p_arr = GMM_total.weights_
    print(p_arr)
    #iterate through every component
    for n in range(0,n_components):
        mu = mu_arr[n]
        sigma = sigma_arr[n]
        p = p_arr[n]
        relu_sum += relu_gmm(mu, sigma, w, p, b)
    return relu_sum

def all_neuron_activated(x,df,n_components, W, B, gamma):
    #get distribution
    df_distr = df.drop(columns=["y"],axis=1)
    distr = fit_distribution(df_distr,n_components,"GMM")
    activated = []
    for neuron_pos in range(0,W.shape[1]):
        #get parameters of pretrained network
        w = W[:, neuron_pos]
        b = B[neuron_pos]
        neuron_pos+=1
        relu_sum = 0
        #get GMM for missing values
        GMM_total = GMM_missing_generation(x,distr,n_components,gamma,type_of_distribution="GMM")
        #get Parameters from GMM
        mu_arr = GMM_total.means_
        sigma_arr = GMM_total.covariances_
        p_arr = GMM_total.weights_
        #iterate through every component
        for n in range(0,n_components):
            mu = mu_arr[n]
            sigma = sigma_arr[n]
            p = p_arr[n]
            relu_sum += relu_gmm(mu, sigma, w, p, b)
        # print(relu_sum)
        activated.append(relu_sum)
    return np.array(activated)

def iterate_data(X,df,n_components, W, B, gamma, model):
    predictions = []
    df_distr = df.drop(columns=["y"],axis=1)
    distr = fit_distribution(df_distr,n_components,"GMM")
    for index, x in X.iterrows():
        activated = []
        try:
            for neuron_pos in range(0,W.shape[0]):
                #get parameters of pretrained network
                w = W[neuron_pos]
                # print(w)
                b = B[neuron_pos]
                # print(b)
                neuron_pos+=1
                relu_sum = 0
                #get GMM for missing values
                GMM_total = GMM_missing_generation(x,distr,n_components,gamma,type_of_distribution="GMM")
                #get Parameters from GMM
                # exception (type(GMM_total)) ansonsten mit model.predict normal weiter 
                # ohne weitere Berechnungen
                # Auswertung mit jeweils 100 instanzen bei Customer / IRIS auf vollständigen
                mu_arr = GMM_total.means_
                sigma_arr = GMM_total.covariances_
                p_arr = GMM_total.weights_
                #iterate through every component
                for n in range(0,n_components):
                    mu = mu_arr[n]
                    sigma = sigma_arr[n]
                    p = p_arr[n]
                    relu_sum += relu_gmm(mu, sigma, w, p, b)
                activated.append(relu_sum)
    
            hidden_activations = np.array(activated)
            input_data = hidden_activations.tolist()
            # input_data = input_data.reshape(-1, 8)
            # predictions.append(new_model.predict(input_data).tolist())
            predictions.append(model.propagate(input_data, 1).tolist())
        except:
            predictions.append(model.propagate(x, 0).tolist())

    return predictions 


def convert_to_one_hot(predictions):
    one_hot_pred = []
    for i in range(0,len(predictions)):
        arr = predictions[i]
        max_val = max(arr)
        result = [0] * len(arr)
        max_index = arr.index(max_val)
        result[max_index] = 1
        one_hot_pred.append(result)
    return one_hot_pred


In [16]:
#main
if __name__ == "__main__":

    warnings.filterwarnings('ignore')
    
    from sklearn.preprocessing import OneHotEncoder
    from sklearn.preprocessing import LabelEncoder
    sc=StandardScaler()  
    le = LabelEncoder()
    df = pd.read_csv("/content/drive/MyDrive/Lehrstuhl Heinrich/IRIS.csv")
    df.rename(columns = {'species':'y'}, inplace = True)
    df["y"] = le.fit_transform(df['y'])    

    
    
    with open('/content/drive/MyDrive/Lehrstuhl Heinrich/iris_test.npy', 'rb') as f:
      X = np.load(f)
      y = np.load(f)
    
    percent_delete = 0.4
    X_miss = generate_missing_values(X, percent_delete)
    X = pd.DataFrame(X_miss)
    
    model = loadModel('/content/drive/MyDrive/Lehrstuhl Heinrich/model_7_iris_2.pt')

    import random
    gamma = random.gauss(20, 1)
    weights_0 = np.array(model.weights[0])
    bias_0 = np.array(model.biases[0])
    X = X
    n_components = 5
    predictions = iterate_data(X, df ,n_components, weights_0, bias_0, gamma, model)

    
    # Berechne die Anzahl der korrekten Vorhersagen
    y_pred = np.array(convert_to_one_hot(predictions))
    y_true_encode = np.zeros((y.size, y.max() + 1))
    y_true_encode[np.arange(y.size), y] = 1
    y_true = y_true_encode
    # y_true = np.array(y_true)
    print(y_pred)
    print(y_true)
    # Berechnung der Accuracy
    correct_predictions = np.sum(np.all(y_true == y_pred, axis=1))
    total_predictions = len(y_true)
    accuracy = np.round((correct_predictions / total_predictions),2)
    print("After deleting: ",percent_delete * 100, "%, The Accuracy is: ",accuracy)

[[0 1 0]
 [1 0 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [1 0 0]
 [0 1 0]
 [0 0 1]
 [0 1 0]
 [0 1 0]
 [0 0 1]
 [1 0 0]
 [0 1 0]
 [1 0 0]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 0 1]
 [0 1 0]
 [1 0 0]
 [1 0 0]]
[[0. 1. 0.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [1. 0. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [1. 0. 0.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [1. 0. 0.]
 [1. 0. 0.]]
After deleting:  40.0 %, The Accuracy is:  0.87
